#### Задание 2. Проверить насколько хорошо работает NER - в отдельном notebook
данные брать из http://www.labinform.ru/pub/named_entities/
1.	проверить NER из nltk/spacy/deeppavlov
2.	написать свой нер попробовать разные подходы
a.	передаём в сетку токен и его соседей
b.	передаём в сетку только токен
c.	свой вариант
3.	сравнить ваши реализованные подходы на качество

In [1]:
!pip install corus
!pip install -U spacy
!python -m spacy info

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

============================== Info about spaCy ==============================

spaCy version    3.3.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines        en_core_web_sm (3.3.0), en_core_web_md (3.3.0)



In [2]:
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.5 MB 397 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
import corus
from corus import load_ne5

import re
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('ieer')

import spacy
from spacy import displacy
import en_core_web_md

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package ieer to /root/nltk_data...
[nltk_data]   Package ieer is already up-to-date!


In [4]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-06-08 21:45:38--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip.3’

collection5.zip.3   100%[===================>]   1.81M  2.26MB/s    in 0.8s    

2022-06-08 21:45:40 (2.26 MB/s) - ‘collection5.zip.3’ saved [1899530/1899530]



In [5]:
!unzip collection5.zip

Archive:  collection5.zip
replace Collection5/001.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.tx

In [6]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='004',
    text='4 октября назначены очередные выборы Верховного Совета Аджарской АР\r\n\r\nПо распоряжению президента Грузии Михаила Саакашвили 4 октября 2008 года назначены очередные выборы Верховного Совета Аджарской АР. Об этом Новости-Грузия сообщили в пресс-службе администрации президента во вторник.\r\n\r\nВыборы Верховного совета Аджарской автономной республики назначены в соответствии с 241-ой статьей и 4-м пунктом 10-й статьи Конституционного закона Грузии <О статусе Аджарской автономной республики>.',
    spans=[Ne5Span(
         index='T1',
         type='ORG',
         start=37,
         stop=54,
         text='Верховного Совета'
     ), Ne5Span(
         index='T2',
         type='LOC',
         start=55,
         stop=67,
         text='Аджарской АР'
     ), Ne5Span(
         index='T3',
         type='GEOPOLIT',
         start=98,
         stop=104,
         text='Грузии'
     ), Ne5Span(
         index='T4',
         type='PER',
         start=105,
  

### NLTK

In [7]:
document = next(records).text
nltk.pos_tag(nltk.word_tokenize(document))

[('Путин', 'JJ'),
 ('освободил', 'NNP'),
 ('от', 'NNP'),
 ('должности', 'NNP'),
 ('торгпредов', 'NNP'),
 ('в', 'NNP'),
 ('Пакистане', 'NNP'),
 ('и', 'NNP'),
 ('Аргентине', 'NNP'),
 ('Премьер-министр', 'JJ'),
 ('России', 'NNP'),
 ('Владимир', 'NNP'),
 ('Путин', 'NNP'),
 ('освободил', 'NNP'),
 ('от', 'NNP'),
 ('занимаемых', 'NNP'),
 ('должностей', 'NNP'),
 ('торгового', 'NNP'),
 ('представителя', 'NNP'),
 ('РФ', 'NNP'),
 ('в', 'NNP'),
 ('Пакистане', 'NNP'),
 ('Ришата', 'NNP'),
 ('Халикова', 'NNP'),
 (',', ','),
 ('а', 'NNP'),
 ('также', 'NNP'),
 ('российского', 'NNP'),
 ('торгпреда', 'NNP'),
 ('в', 'NNP'),
 ('Аргентине', 'NNP'),
 ('и', 'NNP'),
 ('Парагвае', 'NNP'),
 ('(', '('),
 ('по', 'NNP'),
 ('совместительству', 'NNP'),
 (')', ')'),
 ('Виктора', 'NNP'),
 ('Кокорева', 'NNP'),
 (',', ','),
 ('сообщает', 'NNP'),
 ('сайт', 'NNP'),
 ('российского', 'NNP'),
 ('правительства', 'NNP'),
 ('в', 'NNP'),
 ('понедельник', 'NNP'),
 ('.', '.'),
 ('Согласно', 'VB'),
 ('размещенным', 'JJ'),
 ('на', 'N

In [8]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Виктора Кокорева', 'PERSON'),
 ('Кокорев', 'GPE'),
 ('Путин', 'PERSON'),
 ('Ришата Халикова', 'PERSON'),
 ('России Владимир Путин', 'PERSON'),
 ('Халиков', 'PERSON')}

Вывод: NLTK работает плохо с русским языком

# Свой NER

Немного меняю последовательность, т.к. для deeppavlov придётся даунгрейдить TensorFlow до 1.13, а для своего NER понадобится версия от 2.0

In [9]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk

In [10]:
!pip install razdel

from razdel import tokenize
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
records = load_ne5(dir)
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
       
        result = 'None'        
        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
                
    
        words.append([token.text, result])
    words_docs.extend(words)

In [12]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [13]:
df_words['tag'].value_counts()

None        219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [14]:
df_words.head(30)

,word,tag
0,4,None
1,октября,None
2,назначены,None
3,очередные,None
4,выборы,None
5,Верховного,ORG
6,Совета,ORG
7,Аджарской,LOC
8,АР,LOC
9,По,None


In [15]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [16]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [17]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [20]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [21]:
def custom_standardization(input_data):
        return input_data

def data_prep(train_data, seq_len=1, vocab_size = 30000):    
    
    vocab_size = 30000
    #seq_len = 1

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [22]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gPool = tf.keras.layers.GlobalMaxPooling1D()
        self.fc1 = tf.keras.layers.Dense(300, activation='relu')
        self.fc2 = tf.keras.layers.Dense(50, activation='relu')
        self.fc3 = tf.keras.layers.Dense(len(df_words['tag'].value_counts()), activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        return self.fc3(concat_x)

In [40]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 4.5 kB/s 
     |████████████████████████████████| 1.6 MB 44.2 MB/s 
     |████████████████████████████████| 15.7 MB 330 kB/s 
     |████████████████████████████████| 5.8 MB 35.1 MB/s 
     |████████████████████████████████| 438 kB 47.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.0
    Uninstalling numpy-1.18.0:
      Successfully uninstalled numpy-1.18.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 4.2 kB/s 


In [66]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score


class Metrics(Callback):

    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(
            self.model.validation_data[0]))).round()
        val_targ = self.model.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(f" — val_f1: {_val_f1} — val_precision: {_val_precision} — val_recall _val_recall")
        return


metrics_try = Metrics()

Передаём в сетку только токен (seq_len = 1)

Сделаем 1 эпоху для ускорения процесса

In [68]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 1, vocab_size = vocab_size)


model = modelNER()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
hist = model.fit(train_data, validation_data=valid_data, epochs=1)

12444/12444 [==============================] - 317s 25ms/step - loss: 0.2847 - accuracy: 0.9175 - val_loss: 0.2009 - val_accuracy: 0.9386


Передаём в сетку токен и соседей (seq_len = 3)

In [69]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 3, vocab_size = vocab_size)


model = modelNER()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
hist = model.fit(train_data, validation_data=valid_data, epochs=1)

12444/12444 [==============================] - 266s 21ms/step - loss: 0.2949 - accuracy: 0.9139 - val_loss: 0.2028 - val_accuracy: 0.9380


Передаём в сетку токен и несколько соседей (seq_len = 5)

In [70]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 5, vocab_size = vocab_size)


model = modelNER()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
hist = model.fit(train_data, validation_data=valid_data, epochs=1)

12444/12444 [==============================] - 257s 21ms/step - loss: 0.2954 - accuracy: 0.9137 - val_loss: 0.2045 - val_accuracy: 0.9374


Вывод: от числа соседей точность практически не зависит

### Spacy

In [9]:
nlp = en_core_web_md.load()
doc = nlp(document)

displacy.render(doc, jupyter=True, style="dep")
displacy.render(doc, jupyter=True, style="ent")

Spacy работает получше, но я ожидал большего :(

### Deeppavlov

In [10]:
!pip install fasttext
import fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
# установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

Found existing installation: tensorflow 1.13.1
Uninstalling tensorflow-1.13.1:
  Successfully uninstalled tensorflow-1.13.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pydantic-1.3-cp37-cp37m-manylinux2010_x86_64.whl (7.3 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.0.17 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.3 which is incompatible.
spacy 3.3.1 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.3 which is incompatible.


In [12]:
import deeppavlov
from deeppavlov import configs, build_model

In [13]:
!pip install tensorflow==1.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (92.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires numpy>=1.18.5, but you have numpy 1.18.0 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.13.1 which is incompatible.


In [14]:
ner_dp_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
result_1 = ner_dp_model([rus_document])

for i in range(len(result_1[0][0])):
     if result_1 [1][0][i] != 'O':
         print(result_1[0][0][i], result_1[1][0][i])

2022-06-08 21:11:27.496 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2022-06-08 21:11:32.248 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2022-06-08 21:11:33.31 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use standard file APIs to check for files with this prefix.


2022-06-08 21:12:18.557 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model
Нью B-GPE
- I-GPE
Йорк I-GPE
США B-GPE
30 B-DATE
апреля I-DATE
2020 I-DATE
01 B-TIME
: I-TIME
01 I-TIME
США B-GPE
Дональда B-PERSON
Трампа I-PERSON
100 B-MONEY
млн I-MONEY
доз I-MONEY
конца B-DATE
2020 I-DATE
года I-DATE
Bloomberg B-ORG


Deeppavlov - однозначно the best (из здесь рассмотренных).